# Features usados


 - Factor c / v c / s
 - First and last date month 5
 - New Returning , device type (smartphone or computer)
 - Events en pagina
 

In [1]:
import sys #access to system parameters https://docs.python.org/3/library/sys.html

import pandas as pd #collection of functions for data processing and analysis modeled after R dataframes with SQL like features

import matplotlib #collection of functions for scientific and publication-ready visualization

import numpy as np #foundational package for scientific computing

import scipy as sp #collection of functions for scientific computing and advance mathematics

import IPython
from IPython import display #pretty printing of dataframes in Jupyter notebook

import sklearn #collection of machine learning algorithms

#misc libraries
import random
import time
import datetime as dt

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

import featuretools as ft
from sklearn.feature_extraction.text import CountVectorizer


-------------------------


In [2]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
import xgboost as xgb

#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn.model_selection import train_test_split

from sklearn import metrics

#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#Configure Visualization Defaults
#%matplotlib inline = show plots in Jupyter Notebook browser
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8



In [3]:
data_raw = pd.read_csv('events_up_to_01062018.csv')
data_val = pd.read_csv('labels_training_set.csv')

In [4]:
df_labels = data_val.copy(deep=True) 
df = data_raw.copy(deep = True)

In [5]:
pd.set_option('display.max_columns', 23)

In [6]:
df.describe(include= 'all')

,timestamp,event,person,url,sku,model,condition,storage,color,skus,search_term,staticpage,campaign_source,search_engine,channel,new_vs_returning,city,region,country,device_type,screen_resolution,operating_system_version,browser_version
count,2341681,2341681,2341681,191131,1.320530e+06,1321513,1320530,1320530,1320530,505949,113763,11201,191286,106406,204069,204069,204069,204069,204069,204069,204066,204069,204069
unique,1490912,11,38829,248,NaN,208,5,8,63,52267,10964,14,23,4,7,2,2206,122,51,4,393,131,366
top,2018-05-31 01:59:16,viewed product,c76b8417,/,NaN,iPhone 6,Bom,16GB,Preto,"2820,6706,6720,2750,6649,7251,6663,12604,7224,...",Iphone,CustomerService,google,Google,Paid,Returning,Unknown,Sao Paulo,Brazil,Smartphone,360x640,Windows 7,Chrome 66.0
freq,14,1248124,4438,64187,NaN,107262,547617,442096,314925,2606,2577,5239,123354,105195,91753,165827,36866,57304,197699,103502,73234,46648,57953
mean,NaN,NaN,NaN,NaN,6.899178e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,4.028042e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,7.100000e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,2.929000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,7.057000e+03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,1.001400e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
persons = (df.drop_duplicates('person'))['person'].to_frame()
persons_to_train = df_labels['person'].to_frame()
persons_to_predict = persons.loc[~persons['person'].isin(persons_to_train['person'])]


print(persons_to_train.shape)
print(persons_to_predict.shape)

(19414, 1)
(19415, 1)


# Feature cantidad de meses que se metieron
 - Al juntarlo no rindio y bajo un poco

In [8]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['month'] = df['timestamp'].dt.month
df_month = df

In [9]:
df_month.shape

(2341681, 24)

In [10]:
df_month = df_month.groupby('person').agg({'month':'nunique'}).reset_index()

In [11]:
df_month['month'].value_counts()

1    31288
2     4796
3     1828
4      672
5      245
Name: month, dtype: int64

In [12]:
df_month.head()

,person,month
0,0008ed71,1
1,00091926,1
2,00091a7a,1
3,000ba417,1
4,000c79fe,1


# Feature si se metieron despues de comprar
 - Worthless
 

In [13]:
ppl_converted = (df.loc[df['event'] == 'conversion'])['person'].drop_duplicates().to_frame()

In [14]:
ppl_converted.head()

,person
195,49c19e32
724,39df97e0
744,380c0e60
1198,35ee0cc8
2124,c5a0bc36


In [15]:
df_conversion = df.loc[df['person'].isin(ppl_converted['person'])]

In [16]:
df_conversion['timestamp'] = pd.to_datetime(df_conversion['timestamp'])
df_conversion = df_conversion.sort_values(by='timestamp')
df_conversion['day'] = df_conversion['timestamp'].dt.day
df_conversion['month'] = df_conversion['timestamp'].dt.month

In [17]:
df_tiempos = (df_conversion.groupby(['person','month','day','event'])['timestamp']).apply(list).reset_index()
df_tiempos.loc[df_tiempos['person']=='fff2bdde']

,person,month,day,event,timestamp
144888,fff2bdde,5,6,checkout,[2018-05-06 19:39:51]
144889,fff2bdde,5,6,conversion,[2018-05-06 19:53:09]
144890,fff2bdde,5,6,generic listing,"[2018-05-06 19:32:40, 2018-05-06 19:54:08]"
144891,fff2bdde,5,6,searched products,"[2018-05-06 19:35:46, 2018-05-06 19:57:02, 201..."
144892,fff2bdde,5,6,viewed product,"[2018-05-06 19:34:52, 2018-05-06 19:37:19, 201..."
144893,fff2bdde,5,6,visited site,[2018-05-06 19:32:35]
144894,fff2bdde,5,9,checkout,"[2018-05-09 01:11:50, 2018-05-09 02:51:56]"
144895,fff2bdde,5,9,generic listing,[2018-05-09 01:01:04]
144896,fff2bdde,5,9,search engine hit,[2018-05-09 01:00:57]
144897,fff2bdde,5,9,searched products,"[2018-05-09 01:03:33, 2018-05-09 01:05:25]"


In [18]:
df_tiempos = df_tiempos.loc[df_tiempos['event'].isin(['conversion','viewed product'])]

In [19]:
df_labels.loc[df_labels['person']== '0019e639']

,person,label
11105,0019e639,0


In [20]:
df_dsp_d_conv = df_tiempos.sort_values(by=['person','month','day','event'],ascending = [False, False , False, True]).drop_duplicates(subset='person',keep='first')

In [21]:
df_dsp_d_conv['dsp_de_comprar'] = df_dsp_d_conv['event'] == 'viewed product'

In [22]:
df_dsp_d_conv.loc[df_dsp_d_conv['person']=='fff2bdde']

,person,month,day,event,timestamp,dsp_de_comprar
144903,fff2bdde,5,17,viewed product,"[2018-05-17 22:36:25, 2018-05-17 22:38:09, 201...",True


In [23]:
df_dsp_d_conv = df_dsp_d_conv[['person', 'dsp_de_comprar']].reset_index(drop=True)
df_dsp_d_conv.head()

,person,dsp_de_comprar
0,fff72025,False
1,fff2bdde,True
2,ffed0342,True
3,ffe53446,False
4,ffdafc72,False


In [24]:
dsp_d_conv = persons.merge(df_dsp_d_conv, on='person' , how='left')

In [25]:
dsp_d_conv = dsp_d_conv.fillna(False)

## XGboost entrenamiento

In [201]:
df_train = df_labels.merge(dsp_d_conv , left_on='person', right_on='person' , how='inner')

In [202]:
df_train.head()

,person,label,dsp_de_comprar
0,0566e9c1,0,False
1,6ec7ee77,0,False
2,abe7a2fb,0,False
3,34728364,0,False
4,87ed62de,0,False


In [203]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,dsp_de_comprar
0,False
1,False
2,False
3,False
4,False


## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [29]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)



Este es el arbol con sus hiperparametros

In [30]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [31]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [32]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.5454002711395234

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

# Feature Cantidad de dias que vio un mismo modelo

In [33]:
df['day'] = df['timestamp'].dt.day

In [34]:
df_model_viewed = df.loc[df['event'] == 'viewed product']
df_model_viewed['count']= 1

In [35]:
df_model_viewed = df_model_viewed.groupby(['person','month','day', 'model']).agg({'count':'sum'}).reset_index()
df_model_viewed.head()

,person,month,day,model,count
0,00091926,5,3,Motorola Moto X Style,2
1,00091926,5,3,Samsung Galaxy A7 2017,1
2,00091926,5,3,iPhone 6S,1
3,00091926,5,3,iPhone 7,2
4,00091926,5,5,Motorola Moto X Style,2


In [36]:
df_model_viewed['days'] = 1
df_model_viewed = df_model_viewed.groupby(['person', 'model']).agg({'count':'sum','days':'sum'}).reset_index()

In [37]:
df_model_viewed = df_model_viewed.sort_values(by=['count'],ascending = [False]).drop_duplicates(subset='person',keep='first')
df_model_viewed.head()

,person,model,count,days
152999,b1f4dbf6,iPhone 5s,832,39
20317,171e75cb,iPhone 6S,679,31
97033,6f19cfd9,iPhone 6S Plus,622,29
93667,6abd2bf1,iPhone 6,603,47
134690,9bf968c5,iPhone 6,558,32


In [38]:
features = df_model_viewed.drop(['model','count'],axis=1)

In [39]:
features = features.merge(persons, on='person', how = 'right')

In [40]:
features.shape

(38829, 2)

## Mean


In [289]:
df_model_viewed_mean = df.loc[df['event'] == 'viewed product']
df_model_viewed_mean['count']= 1

In [290]:
df_model_viewed_mean = df_model_viewed_mean.groupby(['person','month','day', 'model']).agg({'count':'sum'}).reset_index()
df_model_viewed_mean.head()

,person,month,day,model,count
0,00091926,5,3,Motorola Moto X Style,2
1,00091926,5,3,Samsung Galaxy A7 2017,1
2,00091926,5,3,iPhone 6S,1
3,00091926,5,3,iPhone 7,2
4,00091926,5,5,Motorola Moto X Style,2


In [291]:
df_model_viewed_mean['days'] = 1
df_model_viewed_mean = df_model_viewed_mean.groupby(['person', 'model']).agg({'count':'sum','days':'sum'}).reset_index()

In [293]:
df_model_viewed_mean = df_model_viewed_mean.groupby(['person']).agg({'count':['std','mean'],'days':['std','mean','max']})
df_model_viewed_mean.head()

count                 days              
                std       mean       std      mean max
person                                                
00091926  19.442222  10.333333  3.367916  2.500000  16
00091a7a   0.000000   1.000000  0.000000  1.000000   1
000ba417   7.295626   5.884615  0.561591  1.346154   3
000c79fe        NaN   3.000000       NaN  1.000000   1
000e4d9e  23.233732   9.162162  1.675203  1.837838   9

## XGboost entrenamiento

In [112]:
df_train = df_labels.merge(df_viewed , left_on='person', right_on='person' , how='inner')

In [113]:
df_train.head()

,person,label,"(count, sum)","(count, std)","(count, mean)"
0,0566e9c1,0,23.0,12.020815,11.500000
1,6ec7ee77,0,NaN,NaN,NaN
2,abe7a2fb,0,31.0,3.778595,3.444444
3,34728364,0,24.0,12.124356,8.000000
4,87ed62de,0,9.0,4.949747,4.500000


In [114]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,"(count, sum)","(count, std)","(count, mean)"
0,23.0,12.020815,11.500000
1,NaN,NaN,NaN
2,31.0,3.778595,3.444444
3,24.0,12.124356,8.000000
4,9.0,4.949747,4.500000


## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [115]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 6,
                subsample = 0.8,
                gamma = 1,
                n_estimators = 10)



Este es el arbol con sus hiperparametros

In [116]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [117]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [118]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.6171728315928597

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

# Campaign ppl


In [54]:
(df.loc[df['event'] == 'ad campaign hit']).drop_duplicates('person').shape

(31900, 25)

In [55]:
df_campaign = (df.loc[df['event'] == 'ad campaign hit'])
df_campaign['count'] = 1

In [56]:
df_campaign = df_campaign.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [57]:
df_campaign = df_campaign.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [58]:
df_campaign.head()

person count                    
              sum       std      mean
0  00091926    15  0.504525  1.363636
1  00091a7a     1       NaN  1.000000
2  000ba417     1       NaN  1.000000
3  000c79fe     1       NaN  1.000000
4  000e4d9e    19  8.386497  6.333333

# Visited ppl


In [77]:
df['event'].value_counts()

viewed product       1248124
brand listing         216312
visited site          204069
ad campaign hit       191388
generic listing       160176
searched products     130616
search engine hit     106406
checkout               65315
staticpage             11201
conversion              7091
lead                     983
Name: event, dtype: int64

In [78]:
print((df.loc[df['event'] == 'viewed product']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'visited site']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'ad campaign hit']).drop_duplicates('person').shape)

print((df.loc[df['event'] == 'checkout']).drop_duplicates('person').shape)



(37130, 25)
(38242, 25)
(31900, 25)
(32833, 25)


In [528]:
(df.loc[df['event'] == 'visited site']).drop_duplicates('person').shape

(38242, 25)

In [529]:
df_visited = (df.loc[df['event'] == 'visited site'])
df_visited['count'] = 1

In [530]:
df_visited = df_visited.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [531]:
df_visited = df_visited.groupby('person').agg({'count':['sum' , 'std','mean', 'max']}).reset_index()

In [532]:
df_visited.head()

person count                        
              sum       std      mean max
0  0008ed71     2       NaN  2.000000   2
1  00091926    34  1.100964  1.545455   6
2  00091a7a     1       NaN  1.000000   1
3  000ba417     6  1.000000  2.000000   3
4  000c79fe     1       NaN  1.000000   1

# Checkout ppl

In [508]:
(df.loc[df['event'] == 'checkout']).drop_duplicates('person').shape

(32833, 25)

In [509]:
df_checkout = (df.loc[df['event'] == 'checkout'])
df_checkout['count'] = 1

In [510]:
df_checkout = df_checkout.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [511]:
df_checkout = df_checkout.groupby('person').agg({'count':['sum' , 'std','mean', 'max']}).reset_index()

In [512]:
df_checkout.head()

person count              
              sum  std mean max
0  0008ed71     3  NaN  3.0   3
1  00091926     2  0.0  1.0   1
2  000ba417     6  1.0  2.0   3
3  000c79fe     1  NaN  1.0   1
4  000e4d9e     1  NaN  1.0   1

In [513]:
df_checkout = persons.merge(df_checkout, on='person', how='left')

In [514]:
df_checkout.shape

(38829, 5)

# Viewed ppl

In [105]:
(df.loc[df['event'] == 'viewed product']).drop_duplicates('person').shape

(37130, 25)

In [106]:
df_viewed = (df.loc[df['event'] == 'viewed product'])
df_viewed['count'] = 1

In [107]:
df_viewed = df_viewed.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [108]:
df_viewed = df_viewed.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [109]:
df_viewed.head()

person count                      
              sum        std       mean
0  00091926   372  18.899735  18.600000
1  00091a7a     3        NaN   3.000000
2  000ba417   153  16.093477  51.000000
3  000c79fe     3        NaN   3.000000
4  000e4d9e   339  49.401417  37.666667

In [110]:
df_viewed = persons.merge(df_viewed, on='person', how='left')

In [111]:
df_viewed.shape

(38829, 4)

# Ad ppl

In [121]:
(df.loc[df['event'] == 'ad campaign hit']).drop_duplicates('person').shape

(31900, 25)

In [122]:
df_ad = (df.loc[df['event'] == 'ad campaign hit'])
df_ad['count'] = 1

In [123]:
df_ad = df_ad.groupby(['person','month', 'day']).agg({'count':'sum'}).reset_index()


In [124]:
df_ad = df_ad.groupby('person').agg({'count':['sum' , 'std','mean']}).reset_index()

In [125]:
df_ad.head()

person count                    
              sum       std      mean
0  00091926    15  0.504525  1.363636
1  00091a7a     1       NaN  1.000000
2  000ba417     1       NaN  1.000000
3  000c79fe     1       NaN  1.000000
4  000e4d9e    19  8.386497  6.333333

In [126]:
df_ad = persons.merge(df_ad, on='person', how='left')

In [127]:
df_ad.shape

(38829, 4)

## Juntandolos

In [820]:
features =df_checkout
#features = features.merge(df_ad, on='person', how='left')
features = features.merge(df_visited, on='person', how='left')
#features = features.merge( df_viewed, on='person', how='left')


print(features.shape)

#features.columns = ['person','sum1','std1', 'mean1','sum2','std2', 'mean2','sum3','std3', 'mean3']
features.head()

(38829, 9)


,person,"('count', 'sum')_x","('count', 'std')_x","('count', 'mean')_x","('count', 'max')_x","(count_y, sum)","(count_y, std)","(count_y, mean)","(count_y, max)"
0,4886f805,1.0,NaN,1.000000,1.0,1.0,NaN,1.000000,1.0
1,ad93850f,1.0,NaN,1.000000,1.0,5.0,0.500000,1.250000,2.0
2,0297fc1e,7.0,0.408248,1.166667,2.0,95.0,0.830807,1.610169,5.0
3,2d681dd8,1.0,NaN,1.000000,1.0,2.0,0.000000,1.000000,1.0
4,cccea85e,1.0,NaN,1.000000,1.0,22.0,0.834847,1.833333,3.0


In [821]:
some = pd.read_csv('features_nachota.csv')
features = features.merge(some, on='person', how='left')

In [822]:
features = features.drop('Unnamed: 0', axis=1)

In [826]:
diff = pd.read_csv('diff.csv')
features = features.merge(diff, on='person', how='left')

In [823]:
features = features.merge(df_model_viewed_mean, on='person', how='left')
features.head()

,person,"('count', 'sum')_x","('count', 'std')_x","('count', 'mean')_x","('count', 'max')_x","(count_y, sum)","(count_y, std)","(count_y, mean)","(count_y, max)",total_cs_factor,total_cv_factor,...,WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,"(count, std)","(count, mean)","(days, std)","(days, mean)","(days, max)"
0,4886f805,1.0,NaN,1.000000,1.0,1.0,NaN,1.000000,1.0,0.0,0.048309,...,4.0,4.0,0.0,1.0,0.0,1.0,NaN,4.000000,NaN,1.000000,1.0
1,ad93850f,1.0,NaN,1.000000,1.0,5.0,0.500000,1.250000,2.0,0.0,0.134012,...,1.0,0.0,0.0,5.0,1.0,4.0,8.962886,6.666667,1.154701,1.666667,3.0
2,0297fc1e,7.0,0.408248,1.166667,2.0,95.0,0.830807,1.610169,5.0,0.0,4.260824,...,0.0,2.0,0.0,95.0,1.0,94.0,58.825974,26.933333,12.528824,6.600000,48.0
3,2d681dd8,1.0,NaN,1.000000,1.0,2.0,0.000000,1.000000,1.0,0.0,0.152395,...,6.0,4.0,1.0,2.0,1.0,1.0,2.081666,4.333333,0.000000,1.000000,1.0
4,cccea85e,1.0,NaN,1.000000,1.0,22.0,0.834847,1.833333,3.0,0.0,9.629172,...,3.0,0.0,1.0,22.0,1.0,21.0,35.422433,12.964912,2.078352,2.578947,10.0


In [824]:
features.columns= ['person', 'sum_checkout', 'std_checkout','mean_checkout','max_checkout','sum_visited', 'std_visited','mean_visited','max_visited', 'total_cs_factor', 'total_cv_factor', 'diferencia 5', 'DAY(last month 5)',     'DAY(first month 5)','WEEKDAY(last month 5)', 'WEEKDAY(first month 5)', 'device_type',    'computer_smartphone','new_vs_returning','newr', 'std_count', 'mean_count', 'std_days','mean_days','max_days']

In [825]:
features = features.drop(['max_visited','max_checkout','std_count' ,'mean_count' , 'std_days' , 'mean_days'], axis = 1)

In [827]:
features.shape

(38829, 22)

In [828]:
features.to_csv('features_0.8141.csv',index=False)

In [829]:
some.head()

,Unnamed: 0,person,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr
0,0,000c79fe,0.435989,0.029996,0.0,29.0,29.0,1.0,1.0,0.0,1.0,0.0,1.0
1,1,000e619d,0.000000,0.332996,2.0,18.0,15.0,4.0,1.0,1.0,5.0,1.0,4.0
2,2,001001be,0.000000,0.322710,5.0,22.0,16.0,1.0,2.0,0.0,3.0,1.0,2.0
3,3,001802e4,0.000000,0.043728,0.0,31.0,31.0,3.0,3.0,0.0,1.0,0.0,1.0
4,4,0019e639,0.000000,2.632312,20.0,22.0,2.0,1.0,2.0,1.0,19.0,1.0,18.0


## XGboost entrenamiento

In [830]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(X) 
X.loc[:,:] = scaled_values

In [831]:
df_train = df_labels.merge(features , left_on='person', right_on='person' , how='inner')

In [832]:
df_train.head()

,person,label,sum_checkout,std_checkout,mean_checkout,sum_visited,std_visited,mean_visited,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,max_days,diff_mean,diff_max,session_count
0,0566e9c1,0,1.0,NaN,1.0,17.0,1.059350,1.7,0.0,0.232934,8.0,31.0,22.0,3.0,1.0,0.0,17.0,1.0,16.0,2.0,0.059618,0.247778,8.0
1,6ec7ee77,0,NaN,NaN,NaN,1.0,NaN,1.0,0.0,0.000000,0.0,28.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN
2,abe7a2fb,0,1.0,NaN,1.0,22.0,1.183216,2.0,0.0,0.331879,27.0,29.0,2.0,1.0,2.0,0.0,22.0,1.0,21.0,6.0,0.095119,0.346944,21.0
3,34728364,0,NaN,NaN,NaN,4.0,0.000000,1.0,0.0,0.330539,8.0,27.0,18.0,6.0,4.0,0.0,4.0,1.0,3.0,2.0,0.043148,0.080556,3.0
4,87ed62de,0,2.0,NaN,2.0,1.0,NaN,1.0,0.0,0.106222,0.0,19.0,18.0,5.0,4.0,1.0,1.0,0.0,1.0,2.0,0.155278,0.172222,2.0


In [ ]:
df_train.shape

In [833]:
X, y = df_train.iloc[:,2:],df_train.iloc[:,1]
X.head()

,sum_checkout,std_checkout,mean_checkout,sum_visited,std_visited,mean_visited,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,max_days,diff_mean,diff_max,session_count
0,1.0,NaN,1.0,17.0,1.059350,1.7,0.0,0.232934,8.0,31.0,22.0,3.0,1.0,0.0,17.0,1.0,16.0,2.0,0.059618,0.247778,8.0
1,NaN,NaN,NaN,1.0,NaN,1.0,0.0,0.000000,0.0,28.0,28.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN
2,1.0,NaN,1.0,22.0,1.183216,2.0,0.0,0.331879,27.0,29.0,2.0,1.0,2.0,0.0,22.0,1.0,21.0,6.0,0.095119,0.346944,21.0
3,NaN,NaN,NaN,4.0,0.000000,1.0,0.0,0.330539,8.0,27.0,18.0,6.0,4.0,0.0,4.0,1.0,3.0,2.0,0.043148,0.080556,3.0
4,2.0,NaN,2.0,1.0,NaN,1.0,0.0,0.106222,0.0,19.0,18.0,5.0,4.0,1.0,1.0,0.0,1.0,2.0,0.155278,0.172222,2.0


## Xgboost

Para evaluar usen esta medida que me da valores muy parecidos a los de kaggle, para hacer las predicciones usen el otro

In [1048]:
import xgboost as xgb
model = xgb.XGBClassifier(objective ='reg:linear', 
                colsample_bytree = 1, learning_rate = 0.1,
                max_depth = 7,
                subsample = 0.9,
                gamma = 1,
                n_estimators = 50)



Este es el arbol con sus hiperparametros

In [1049]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [1024]:
xgb_classifier(X_train, X_test, y_train, y_test, useTrainCV=False, cv_folds=5, early_stopping_rounds=50)


XGBoost Classifier
El puntaje auc es: 0.8345368222220962


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=6, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.8)

In [1050]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=1, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.9)

In [1051]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,  model.predict_proba(X_test)[:,1])

0.8413493445570726

In [955]:
X_predict.head()

,person,label,sum1,std1,mean1,sum2,std2,mean2,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,max_days,diff_mean,diff_max,session_count
0,4886f805,0.180970,1.0,NaN,1.000000,1.0,NaN,1.000000,0.000000,0.048309,0.0,18.0,18.0,4.0,4.0,0.0,1.0,0.0,1.0,1.0,0.058056,0.058056,1.0
1,0297fc1e,0.227695,7.0,0.408248,1.166667,95.0,0.830807,1.610169,0.000000,4.260824,26.0,28.0,2.0,0.0,2.0,0.0,95.0,1.0,94.0,48.0,0.120095,0.678889,70.0
2,2d681dd8,0.186071,1.0,NaN,1.000000,2.0,0.000000,1.000000,0.000000,0.152395,9.0,27.0,18.0,6.0,4.0,1.0,2.0,1.0,1.0,1.0,0.170093,0.376389,3.0
3,cccea85e,0.217923,1.0,NaN,1.000000,22.0,0.834847,1.833333,0.000000,9.629172,23.0,31.0,7.0,3.0,0.0,1.0,22.0,1.0,21.0,10.0,0.152070,0.847778,42.0
4,4c8a8b93,0.212943,2.0,0.000000,1.000000,20.0,2.549510,4.000000,0.837551,1.741019,4.0,22.0,18.0,1.0,4.0,0.0,20.0,1.0,19.0,4.0,0.168810,0.754722,21.0


In [956]:
X_predict = persons_to_predict.merge(features, on='person', how='left')
X_predict.head()


,person,label,sum_checkout,std_checkout,mean_checkout,sum_visited,std_visited,mean_visited,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,max_days,diff_mean,diff_max,session_count
0,4886f805,0.178530,1.0,NaN,1.000000,1.0,NaN,1.000000,0.000000,0.048309,0.0,18.0,18.0,4.0,4.0,0.0,1.0,0.0,1.0,1.0,0.058056,0.058056,1.0
1,0297fc1e,0.216133,7.0,0.408248,1.166667,95.0,0.830807,1.610169,0.000000,4.260824,26.0,28.0,2.0,0.0,2.0,0.0,95.0,1.0,94.0,48.0,0.120095,0.678889,70.0
2,2d681dd8,0.183177,1.0,NaN,1.000000,2.0,0.000000,1.000000,0.000000,0.152395,9.0,27.0,18.0,6.0,4.0,1.0,2.0,1.0,1.0,1.0,0.170093,0.376389,3.0
3,cccea85e,0.216263,1.0,NaN,1.000000,22.0,0.834847,1.833333,0.000000,9.629172,23.0,31.0,7.0,3.0,0.0,1.0,22.0,1.0,21.0,10.0,0.152070,0.847778,42.0
4,4c8a8b93,0.220525,2.0,0.000000,1.000000,20.0,2.549510,4.000000,0.837551,1.741019,4.0,22.0,18.0,1.0,4.0,0.0,20.0,1.0,19.0,4.0,0.168810,0.754722,21.0


In [957]:
df_entrie = persons_to_predict
df_entrie['label'] = model.predict_proba(X_predict.iloc[:,2:])[:,1]
df_entrie.head()

,person,label
0,4886f805,0.005215
2,0297fc1e,0.152325
3,2d681dd8,0.023079
4,cccea85e,0.123835
5,4c8a8b93,0.047986


In [958]:
df_entrie.shape

(19415, 2)

Obtengo un resultado con los que separe para el test mas arriba en :
```python
    X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)
    ```
###### Lo que hace es sacar las predicciones para X_test y evaluarlos con y_test
    

In [965]:
df_entrie.to_csv('submit.csv',index= False)

382164     0.0
542883     0.0
829059     0.0
830239     0.0
1223472    0.0
Name: label, dtype: float32

 # Random Forest feature importance
   - Algoritmo usado para sacar importancia de los features y ver cuales no nos estan sirviendo al modelo
 #### Es parecido a lo que hice en xgboost con algunas cosas magicas, que use para ponerlo en df y ver resultados
 
  ## NOTAR :
   - Que use la X de xgboost
   - Y que el codigo abajo del dataframe comentado puede funcionar para separar los df pidiendo que nos deje las columnas esas

In [415]:
X = X.fillna(0)

In [416]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np
#Load boston housing dataset as an example

names = X.columns
rf = RandomForestRegressor()
rf.fit(X, y)
zipped = zip(map(lambda x: round(x, 4), rf.feature_importances_), names)
feature = sorted(zipped, key=lambda x: x[1])

In [417]:
feat_importance = pd.DataFrame(feature, columns=['importance', 'feature'])
feat_importance.sort_values('importance', ascending=False).head(100)

,importance,feature
7,0.0956,diff_max
8,0.0946,diff_mean
27,0.0921,total_cv_factor
13,0.0610,mean_count
21,0.0544,std_count
25,0.0528,sum3
0,0.0463,DAY(first month 5)
10,0.0425,mean1
23,0.0411,sum1
17,0.0370,session_count


In [418]:
feat_importance =feat_importance.loc[feat_importance['importance'].isin(feat_importance['importance'].nlargest(24))]
feats_servibles = feat_importance['feature'].tolist()
feats_servibles.append('person')
features = features[feats_servibles]

In [387]:
features.head()

,DAY(first month 5),DAY(last month 5),WEEKDAY(first month 5),WEEKDAY(last month 5),computer_smartphone,device_type,diferencia 5,max_days,mean2,mean3,mean_count,mean_days,newr,std1,std_count,std_days,sum1,sum3,total_cs_factor,total_cv_factor,person
0,18.0,18.0,4.0,4.0,1.0,0.0,0.0,1.0,4.000000,1.000000,4.000000,1.000000,1.0,NaN,NaN,NaN,4.0,1.0,0.0,0.048309,4886f805
1,14.0,22.0,0.0,1.0,5.0,0.0,7.0,3.0,6.666667,1.000000,6.666667,1.666667,4.0,7.371115,8.962886,1.154701,20.0,1.0,0.0,0.134012,ad93850f
2,2.0,28.0,2.0,0.0,95.0,0.0,26.0,48.0,7.214286,1.166667,26.933333,6.600000,94.0,5.228295,58.825974,12.528824,404.0,7.0,0.0,4.260824,0297fc1e
3,18.0,27.0,4.0,6.0,2.0,1.0,9.0,1.0,6.500000,1.000000,4.333333,1.000000,1.0,0.707107,2.081666,0.000000,13.0,1.0,0.0,0.152395,2d681dd8
4,7.0,31.0,0.0,3.0,22.0,1.0,23.0,10.0,56.846154,1.000000,12.964912,2.578947,21.0,26.241971,35.422433,2.078352,739.0,1.0,0.0,9.629172,cccea85e


,person,total_cs_factor,total_cv_factor,diferencia 5,DAY(last month 5),DAY(first month 5),WEEKDAY(last month 5),WEEKDAY(first month 5),device_type,computer_smartphone,new_vs_returning,newr,month,days
0,000c79fe,0.435989,0.029996,0.0,29.0,29.0,1.0,1.0,0.0,1.0,0.0,1.0,1,1.0
1,000e619d,0.000000,0.332996,2.0,18.0,15.0,4.0,1.0,1.0,5.0,1.0,4.0,1,2.0
2,001001be,0.000000,0.322710,5.0,22.0,16.0,1.0,2.0,0.0,3.0,1.0,2.0,1,1.0
3,001802e4,0.000000,0.043728,0.0,31.0,31.0,3.0,3.0,0.0,1.0,0.0,1.0,1,1.0
4,0019e639,0.000000,2.632312,20.0,22.0,2.0,1.0,2.0,1.0,19.0,1.0,18.0,4,5.0


In [629]:
X_predict.columns

Index(['person', 'label', 'sum1', 'std1', 'mean1', 'sum2', 'std2', 'mean2',
       'total_cs_factor', 'total_cv_factor', 'diferencia 5',
       'DAY(last month 5)', 'DAY(first month 5)', 'WEEKDAY(last month 5)',
       'WEEKDAY(first month 5)', 'device_type', 'computer_smartphone',
       'new_vs_returning', 'newr', 'max_days', 'diff_mean', 'diff_max',
       'session_count'],
      dtype='object')

# 0.84121 SUBMISSION

 ## Features
 
 - factores c/v c/s
 - diferencia entre primer y ultimo dia que se metio en el mes 5
 - primer dia del mes 5 que se metio, ultimo dia del mes 5 y que numero de dia de la semana fue
 - device type que tipo fue, y cuantas veces se metio en ese
 - new or returning y cuantas veces lo fueron
 - cuantos meses distintos se metieron
 - dias distintos que vio un modelo 
 - checkout y visited site, promedio, suma, std de todos los dias

'person', 'label', 'sum1', 'std1', 'mean1', 'sum2', 'std2', 'mean2',
       'total_cs_factor', 'total_cv_factor', 'diferencia 5',
       'DAY(last month 5)', 'DAY(first month 5)', 'WEEKDAY(last month 5)',
       'WEEKDAY(first month 5)', 'device_type', 'computer_smartphone',
       'new_vs_returning', 'newr', 'max_days', 'diff_mean', 'diff_max',
       'session_count'